
# PyShop Session 4
***
##Advanced Topics in Statistics and Data Visualization

## Outline

1.  Posting forms using requests
2.  Downloading a single PSID file
3.  Reading in the sas (hand waving!)  Just use PSIDpy
4.  Loading the fam_vars file and the ind_vars file
5.  Cleaning up the data a bit
6.  Merge, concatenate, join, groupby, etc.
7.  Indices, heirarchical indices, loc, idx, etc.  How to find data in your dataframe
8.  Descriptive statistics and plotting
9.  OLS with StatsModels
10.  Plotting with SeaBorn
11.  Adjusting your matplotlibrc file

## Posting Forms Using Requests

* #### Requests makes http possible
* #### A request returns a "response" object
* #### What's in a respons?  Let's check it out!


In [ ]:
#Just to avoid problems later, import everything now
import zipfile
import tempfile
import os
import requests
import shutil
import getpass
#import seaborn.apionly
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from scipy import stats
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from psid_py import read_sas
from io import BytesIO

%matplotlib inline

In [ ]:
#Create a requests session object, which will do all the HTTP for you
c = requests.Session()

#Define the URL of the login page
URL = 'http://simba.isr.umich.edu/u/login.aspx'

#Call the login page to "get" its raw HTML data
page = c.get(URL)

In [ ]:
# Return the content in unicode (string)
page.text

In [ ]:
# Return the content in bytes
page.content

In [ ]:
# Retrieve json... if any!
page.json()

In [ ]:
# Retrieve the cookie jar
page.cookies

In [ ]:
# Retrieve the headers
page.headers

In [ ]:
# Return the links in the header
page.links

In [ ]:
# Return the url
page.url

###What do we need this object for?
* #### To post forms, you need form variables
* #### We will scrape the html for the variables we want

1. Navigate to the login page, right-click on the login window, and click "Inspect Element".  This will open a side bar showing the html and some other stuff.  

2. At the top, there will be several tabs; select the 'Network' tab.  

3. Now, click the 'Clear' button, right next to the red dot at the top.  

4. Now login.  At this point you'll see a bunch of things show up in the table.  

5. Find anything related to login.  For the PSID website this is 'Login.aspx'.  Click this and select the 'Headers' tab on the lower part of the side bar.

6. Now scroll down to the 'Form Data' section.

All of the variables listed here will be submitted to the form.  We are going to need to scrape the html for the values and submit them with our login information.

Here's how we'll scrape the page using Beautiful Soup:

In [ ]:
soup = BeautifulSoup(page.content)
viewstate = soup.findAll("input", {"type": "hidden",
                         "name": "__VIEWSTATE"})
radscript = soup.findAll("input", {"type": "hidden",
                         "name": "RadScriptManager1_TSM"})
eventtarget = soup.findAll("input", {"type": "hidden",
                           "name": "__EVENTTARGET"})
eventargument = soup.findAll("input", {"type": "hidden",
                             "name": " __EVENTARGUMENT"})
viewstategenerator = soup.findAll("input", {"type": "hidden",
                                  "name": "__VIEWSTATEGENERATOR"})
eventvalidation = soup.findAll("input", {"type": "hidden",
                               "name": "__EVENTVALIDATION"})
radscript = soup.findAll("input", {"type": "hidden", "name":
                         "RadScriptManager1_TSM"})


###What is beautiful soup doing!?
* #### When you 'make the soup', BS4 converts the html into a tree of objects using an html parser
* #### 4 types of objects in the soup you might deal with:
    1. tag: this corresponds to the tag in the original html file (usually marked by 'class')
    2. NavigableString: text found within the tag, eg. link text
    3. BeautifulSoup: the document itself
    4. Comments: exactly what it sounds like! introduced by '%' in html
* #### BS4 offers many options for navigating the tree, but for our purpuses 'findAll' will suffice
* #### In our example, we are looking for 'input' tag with the attributed in the dictionary

In [ ]:
print(viewstate)
print(eventtarget)

In [ ]:
#First, define a username and password
#NOTE: Please don't abuse my account... I don't want to be barred from the PSID!
#USERNAME = input("Please enter your PSID username: ")
#PASSWORD = getpass.getpass("Please enter your PSID"
#                                       + " password: ")

USERNAME = "tyler.abbot@sciencespo.fr"
PASSWORD = "tyler.abbot"

In [ ]:
#Gather form data into a single dictionary
params = {'RadScriptManager1_TSM': radscript[0]['value'],
          '__EVENTTARGET': '',
          ' __EVENTARGUMENT': '',
          '__VIEWSTATE': viewstate[0]['value'],
          '__VIEWSTATEGENERATOR': viewstategenerator[0]['value'],
          '__EVENTVALIDATION': eventvalidation[0]['value'],
          'ctl00$ContentPlaceHolder1$Login1$UserName': USERNAME,
          'ctl00$ContentPlaceHolder1$Login1$Password': PASSWORD,
          'ctl00$ContentPlaceHolder1$Login1$LoginButton': 'Log In',
          'ctl00_RadWindowManager1_ClientState': ''}


In [ ]:
#Post the login form.  NOTE: Response code 200 implies OK
c.post('http://simba.isr.umich.edu/u/Login.aspx', data=params,
       headers={"Referer": "http://psidonline.isr.umich.edu/"})

###Note the 'post' and 'headers'.  Need to tell the site where we are coming from.

In [ ]:
#Create a temporary directory to store unzipped files
#temp_dir = tempfile.mkdtemp() + os.sep

#File names in the psid are numbered
file = "1056"
url = 'http://simba.isr.umich.edu/Zips/GetFile.aspx?file=' + file
data1 = c.get(url, allow_redirects=False)

#Download a second file
file = "1058"
url = 'http://simba.isr.umich.edu/Zips/GetFile.aspx?file=' + file
data2 = c.get(url, allow_redirects=False)

In [ ]:
data1.content

### To extract the sas data, we'll use the 'psid_py' package (because SAS is not supported by Pandas)

In [ ]:
#Create a temporary directory to store unzipped files
temp_dir = tempfile.mkdtemp() + os.sep

x = pd.DataFrame()
y = pd.DataFrame()
frames = [x, y]

for i, data in enumerate([data1, data2]):
    #Extract the zipped files
    zipped = zipfile.ZipFile(BytesIO(data.content))
    files_to_unzip = (x for x in zipped.namelist() if 
                      any(['.sas' in x, '.txt' in x]))

    for NAME in files_to_unzip:
        temp_name = zipped.extract(NAME, temp_dir)
        #Test if you have just found the dictionary
        if temp_name.find('.sas') >= 0:
            dict_file = str(temp_name)
        #If not, you have found the data
        else:
            data_file = str(temp_name)

    #Use psidPy to read the sas file in
    frames[i] = pd.concat([frames[i],
                           read_sas.read_sas(data_file, dict_file)])

#Remove the temporary directory
shutil.rmtree(temp_dir)

###We now have two data frames containing our psid data

In [ ]:
x = pd.DataFrame(frames[0])
y = pd.DataFrame(frames[1])
print(x.shape)
print(y.shape)

Here is a list of the variables we are going to pull out and their corresponding code in each year:


| Variable               | 1968 | 1969  |
|------------------------|------|-------|
| Family Number          | V3   | V442  |
| Total Food Consumption | V334 | V863  |
| Head Hourly Earn       | V337 | V871  |
| Head Education         | V313 | V794  |
| Wifes Education        | V246 | NA    |

###So, to start we'll just drop all of the unnecessary columns.


In [ ]:
vars68 = ['V3',
          'V334',
          'V337',
          'V313',
          'V246']

vars69 = ['V442',
          'V863',
          'V871',
          'V794']

frame68 = x[vars68].copy()
frame69 = y[vars69].copy()

frame68.head()

###Notice here that I explicitly created a copy.  
###Change the column names to be something a bit more recognizable.

In [ ]:
frame68.columns = ['fam_id',
                   'foodc',
                   'head_hourly_wage',
                   'head_educ',
                   'wife_educ']

frame69.columns = ['fam_id',
                   'foodc',
                   'head_hourly_wage',
                   'head_educ']

In [ ]:
frame68['year'] = 1968
frame69['year'] = 1969

In [ ]:
frame69.head()

##Concatenate, Join, and Merge

* ### There are two functions you can use to do this sort of merge: `join` and `merge`.
* ### Pandas expects you to be specific about exactly what type of merge operation you would like to do.
* ### Be aware of duplicates in your data.

### Concatenate - pooling the two data frames into one.

In [ ]:
# Concatenating the rows together
concat_rows = pd.concat([frame68, frame69])
print(frame68.shape, frame69.shape, concat_rows.shape)

In [ ]:
# Why is there a new columns?
concat_rows

In [ ]:
# Simply append
concat_rows = frame68.append(frame69)

In [ ]:
# Fixing the index
concat_rows = pd.concat([frame68, frame69], ignore_index=True)
concat_rows

In [ ]:
# Concatenate columns NOTE: Watch out for the index!
temp68 = frame68.set_index('fam_id')
temp69 = frame69.set_index('fam_id')
concat_cols = pd.concat([temp68, temp69], axis=1)
concat_cols.head()

In [ ]:
#Fixing column names
concat_cols.columns = ['foodc68',
                       'head_hourly_wage68',
                       'head_educ68',
                       'wife_educ68',
                       'year68',
                       'foodc69',
                       'head_hourly_wage69',
                       'head_educ69', 
                       'year69']
concat_cols.tail()

### Join and Merge

* **one-to-one** - Refers to merging two data frames by finding exact matches of the key columns.
* **many-to-one** - When the key on one object could contain repetitions.  For instance, a column of group means being joined to a DataFrame containing individual observations.
* **many-to-many** - This is the case when you would like to merge several columns on several keys.  In this case, there could even be repition in some of the keys, in which case the merge method will determine which indices to repeat.

Methods:

* **left** - Use key from left frame.
* **right** - Use key from right frame.
* **inner** - Keep only the intersection of keys.
* **outer** - Keep the union of keys.


In [ ]:
# A simple example of left, right, inner, and outer
a = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
b = pd.DataFrame({'C': [1]})

In [ ]:
a

In [ ]:
b

In [ ]:
# left join
a.join(b, how='left')

In [ ]:
# right join
a.join(b, how='right')

In [ ]:
#To make this more interesting...
#b['D'] = 2
#b = b.set_index(b['D'])
# inner join
a.join(b, how='inner')

In [ ]:
# outer join
a.join(b, how='outer')

In [ ]:
temp68 = frame68.set_index('fam_id')
temp69 = frame69.set_index('fam_id')

#This produces an error!
join1 = temp68.join(temp69)

### Join notices the problem of the column names and requires you to specify a suffix

In [ ]:
join1 = temp68.join(temp69, rsuffix='69')
join1.head()

In [ ]:
# Same number of rows as the left frame
print(temp68.shape, join1.shape)

In [ ]:
#Right join drops entries in 68 not present in 68
join1 = temp68.join(temp69, how='right',rsuffix='69')
print(temp69.shape, join1.shape)

#Inner join drops entries not present in both
join1 = temp68.join(temp69, how='inner',rsuffix='69')
print(temp68.shape, temp69.shape, join1.shape)

#Outer join keeps all entries
join1 = temp68.join(temp69, how='outer',rsuffix='69')
print(temp68.shape, temp69.shape, join1.shape)

### Merge accomplishes the same thing, but is more flexible

In [ ]:
#Right merge drops entries in 68 not present in 68
merge1 = pd.merge(temp68.reset_index(), temp69.reset_index(),
                  how='right', on='fam_id', suffixes=('', '69'))\
    .set_index('fam_id')
print(temp69.shape, merge1.shape)

###Problem solving:

* #### Are your DataFrames sorted?
* #### Are there duplicate entries in your indices?
* #### Did you specify the right type of join?  The defualt is an inner join.  Is this what you want?

## Groupby and Heirarchical Indexing

In [ ]:
# Take a concatenation of entries as our data set
data = pd.concat([frame68, frame69], ignore_index=True).set_index('fam_id')
data.head()

* ####Groupby allows you to efficiently group data along almost any dimension
* #### Creates a groupby object that avoids cluttering up the memory and name space
* #### Allows you freedom to apply almost any function to the groups

###We will study average hourly wage by husband's education group

To accomplish this, we will take two steps (there may be more efficient ways to do this, but this is instructive and clear):

1. Generate a vector of education level means using `groupby`.
2. Merge this vector into the `data` DataFrame.

### Groupby works in 3 steps
1. **Split** the data into groups based on criteria.
2. **Apply** a function to each group independently.
3. **Combine** the results into the data structure.

###Split

In [ ]:
grouped = data.groupby('head_educ')
grouped

###Apply

In [ ]:
# Calculate group means
means = grouped.aggregate(np.mean)
means

In [ ]:
# Other things you can do...
# counts
grouped.size()

In [ ]:
# Descriptive statistics
grouped.describe()

In [ ]:
# Sums
sums = grouped.aggregate(np.sum)
sums

In [ ]:
# Or even a custom function
null = grouped.aggregate(lambda x: np.sum(x)*0)
null

In [ ]:
# Return just one column
wage_means = grouped['head_hourly_wage'].aggregate(np.mean)
wage_means

### That was aggregation, can also do transformations and filtering

In [ ]:
#Transforming wage by taking the difference from the mean
meandif = lambda x: x - x.mean()
transformed = grouped['head_hourly_wage'].transform(meandif)
print(data['head_hourly_wage'].head(),transformed.head())

#Filtering out those with very low hourly wage
filtered = grouped['head_hourly_wage'].filter(lambda x: np.mean(x) > 10.)
print(filtered.head())

###Combine - Once you have your means, merge them back into the data set

In [ ]:
#First, rename the column in wage means
wage_means.name = 'avg_wage_educ'
data1 = pd.merge(data.reset_index(), wage_means.reset_index(),
                 how='left', on='head_educ').set_index('fam_id')
data1.head()

In [ ]:
print(data.shape, data1.shape)
print(wage_means)

In [ ]:
# Just to recap, here's what we did:
wage_means = data.groupby('head_educ')['head_hourly_wage']\
    .aggregate(np.mean)
wage_means.name = 'avg_wage_educ'
data2 = pd.merge(data.reset_index(), wage_means.reset_index(),
                 how='left', on='head_educ').set_index('fam_id')
data2.head()

##Indices and Heirarchical Indices

### Selecting data from within the data frame

In [ ]:
# Drop wife_educ, because we are never going to use it
data = data1.copy().drop('wife_educ', axis=1)

# Selecting by column
data['foodc'].head()

In [ ]:
# The same thing
data.foodc.head()

In [ ]:
# Selecting by label
data.loc[1]

In [ ]:
#Combining the two
data.foodc.loc[1]

In [ ]:
# Slicing with loc... requires sorting!
data.sort_index(inplace=True)
data.foodc.loc[1:5]

###Many Pandas methods require this, so you might think about sorting early in your analysis.

In [ ]:
# Can use loc and array like indexing
data.loc[1:5, 'foodc']

In [ ]:
# iloc is for integer labels, but acts on postion 
data.iloc[1:5]

In [ ]:
#Boolean indexing
data[data.foodc > 1000][:10]

But, you'll notice that we only dropped entries where the row had low consumption.  What if we wanted to drop both entries for a family if they had one year of low consumption.  This is where heirarchical indexing can help.

Heirarchical indexing defines several layers of indices.  In our case, the natural indices are `fam_id` and `year`.  So, we can redefine the index as follows, specifying a list of index names instead of a single string:

In [ ]:
data = data.reset_index().set_index(['fam_id', 'year'])
data.head()

Notice now that the `fam_id` index spans several columns.  This is because each entry in `fam_id` is associated to several entries in `year`, which in turn are associated to individual rows.

Now, we can again try the boolean indexing we had before: 

In [ ]:
data[data.foodc > 1000][:10]

But wait!  That didn't fix our problem!  What we need to do is retrieve the primary index of the boolean array that is returned.  What does this mean?  Let's break down the steps of a boolean heirarchical index operation.

First, what does the boolean operation return?

In [ ]:
data.foodc > 1000

In [ ]:
high_food = data.foodc > 1000
high_food.index.get_level_values('fam_id')

It returns a multi-indexed series object containing `True` and `False` values.  We would like to use this to reference the `fam_id` index.  There are several ways we could achieve this using techniques we've already learned:

1. We could use groupby and apply `any` to return a vector for any false entries.
2. We can reindex and 

In [ ]:
data.index.get_level_values('fam_id')

In [ ]:
idx = pd.IndexSlice
high_food = data.foodc > 1000
high_food[data.index.get_level_values('fam_id')]
#data.loc[idx[high_food,:], :].head()
#idx[high_food]

NOTE:  I cant seem to get this to work, so if you want a fun homework assignment, give it a try!

We could also try the groupby method:

In [ ]:
# Huzzah!  I think that worked!
data.groupby(level=0).\
    filter(lambda x: all([v > 1000 for v in x['foodc']])).head()



###Descriptive Statistics and Plotting

In [ ]:
# Pandas comes with built in descriptive statistics
data.describe()

In [ ]:
# Again you can use groupby
data.groupby('head_educ').describe()

In [ ]:
# To deal with plots, drop na
data = data.dropna()

In [ ]:
# Pandas built in plotting
# Note that the styling is seaborn
data.foodc.plot(kind='hist')

In [ ]:
plt.hist(data.foodc.values)

In [ ]:
# Change number of bins
data.foodc.plot(kind='hist', bins=20)

In [ ]:
# Histogram of all the variables... not so much
data.plot(kind='hist', stacked=True, bins=20)

### Seaborn

In [ ]:
# Plotting a distribution over our histogram
sns.distplot(data.foodc)

In [ ]:
#Adding a 'rugplot' to the histogram
sns.distplot(data.foodc, kde=False, rug=True)

In [ ]:
#Kernel density estimation
sns.distplot(data.foodc, hist=False)

In [ ]:
#Using kdeplot to do the same thing, but changing the bandwidth
sns.kdeplot(data.foodc)
sns.kdeplot(data.foodc, bw=0.2)
sns.kdeplot(data.foodc, bw=2)

In [ ]:
#Using distplot to fit a parametric distribution
sns.distplot(data.foodc, kde=False, fit=stats.gamma)

In [ ]:
# seaborn makes bivariate plots really nice
sns.jointplot(x = data.foodc, y = data.head_hourly_wage)

In [ ]:
# Subsetting for outliers
sns.jointplot(x = data.foodc[data.head_hourly_wage < 80],
              y = data.head_hourly_wage[data.head_hourly_wage < 80])

In [ ]:
# Further subset
sns.jointplot(x = data.foodc[(data.head_hourly_wage < 80) &
                             (data.head_hourly_wage > 0) &
                             (data.foodc > 0)],
              y = data.head_hourly_wage[(data.head_hourly_wage < 80) &
                             (data.head_hourly_wage > 0) &
                             (data.foodc > 0)])

In [ ]:
# Kernel dinsity
sns.jointplot(x = data.foodc[(data.head_hourly_wage < 80) &
                             (data.head_hourly_wage > 0) &
                             (data.foodc > 0)],
              y = data.head_hourly_wage[(data.head_hourly_wage < 80) &
                             (data.head_hourly_wage > 0) &
                             (data.foodc > 0)],
              kind = 'kde')

In [ ]:
# Pairwise plot
sns.pairplot(data)

### Subgroup plotting using FacetGrit

In [ ]:
#First, let's generate a subset of data we are interested in
#NOTE: Here I'm resetting the index so I can use it more easily,
#I'm completely ignoring the fact that subsetting the data in
#this way drops observations... don't worry about it, just enjoy
#the pretty pictures.
subset = pd.DataFrame(data[(data.head_hourly_wage < 20) &
                           (data.head_hourly_wage > 0) &
                           (data.foodc > 0)].reset_index())

g = sns.FacetGrid(subset, col='year', hue='head_educ')
g.map(plt.scatter, 'foodc', 'head_hourly_wage')

In [ ]:
#We can also make the dots more transparent using the `alpha`
#argument, which is available for almost all plotting using
#matplotlib
#Here we'll pool all observations together
g = sns.FacetGrid(subset, hue='head_educ')
g.map(plt.scatter, 'foodc', 'head_hourly_wage', alpha = 0.5)

In [ ]:
# Regression plotting
sns.regplot(x = subset.foodc, y = subset.head_hourly_wage)

In [ ]:
# Linear model plotting
# NOTE: lml uses Facet Grid
sns.lmplot(x = 'foodc', y = 'head_hourly_wage', data = subset)

In [ ]:
sns.lmplot(x = 'foodc', y = 'head_hourly_wage', hue='head_educ',
           data = subset)

In [ ]:
# Splitting to columns
sns.lmplot(x = 'foodc', y = 'head_hourly_wage', col = 'head_educ',
           hue='head_educ', data = subset)

In [ ]:
#... which is kind of small, so you could also do it as rows
sns.lmplot(x = 'foodc', y = 'head_hourly_wage', row = 'head_educ',
           hue='head_educ', data = subset)

## A Taste of Statistics Using StatsModles

* ### Written by econoetricians
* ### Works similar to R (in fact patsy allows you to input r like model formulae)
* ### Development is fairly active, but documentation is not so nice

In [ ]:
model = sm.OLS(subset.foodc, subset.head_hourly_wage)
results = model.fit()
print(results.summary())

In [ ]:
# Using dummies for education
X = pd.concat([subset.head_hourly_wage,
               pd.get_dummies(subset.head_educ)], axis = 1)
model = sm.OLS(subset.foodc, X)
results = model.fit()
print(results.summary())

Sadly, we are running out of time, but StatsModels offers many other models to estimate.  These include the following, but the number is still growing:

* Linear regression
* General linear models
* Robust linear models
* Linear mixed effects models
* Discrete models
* Time series models
* Survival and duration models
* Nonparameteric methods
* GMM
* Empirical likelyhood

### Matplotlibrc

The last thing I'd like to mention is the `matplotlibrc` file.  This is loaded when you import matplotlib and sets the default parameters for your plots.  For instance, if you prefer to have a grid on your plots, you can set this automatically.

According to the matplotlib documentation, you should save the file in one of the following places:

* For linux: `~.config/matplotlib/matplotlibrc`
* For other: `.matplotlib/matplotlibrc`

You can also find the system file in one of the following places

* For linux: `/usr/lib/pythonX.X/site-packages/matplotlib/mpl-data/matplotlibrc`
* For windows: `C:\PythonXX\Lib\site-packages\matplotlib\mpl-data\matplotlibrc`

The system file can serve as a template for your own custom one, which should be saved in the former folder.  I'll paste here (well, ater this text) my own so you can see an example (it's quite long).

In this file you can set a plethora of tiny details that might interest you.  I encourage you to take a look and set your own, as it will help you to understand all that matplotlib can do.

In [ ]:
### MATPLOTLIBRC FORMAT

# This is a sample matplotlib configuration file - you can find a copy
# of it on your system in
# site-packages/matplotlib/mpl-data/matplotlibrc.  If you edit it
# there, please note that it will be overwritten in your next install.
# If you want to keep a permanent local copy that will not be
# overwritten, place it in the following location:
# unix/linux:
#     $HOME/.config/matplotlib/matplotlibrc or
#     $XDG_CONFIG_HOME/matplotlib/matplotlibrc (if $XDG_CONFIG_HOME is set)
# other platforms:
#     $HOME/.matplotlib/matplotlibrc
#
# See http://matplotlib.org/users/customizing.html#the-matplotlibrc-file for
# more details on the paths which are checked for the configuration file.
#
# This file is best viewed in a editor which supports python mode
# syntax highlighting. Blank lines, or lines starting with a comment
# symbol, are ignored, as are trailing comments.  Other lines must
# have the format
#    key : val # optional comment
#
# Colors: for the color values below, you can either use - a
# matplotlib color string, such as r, k, or b - an rgb tuple, such as
# (1.0, 0.5, 0.0) - a hex string, such as ff00ff or #ff00ff - a scalar
# grayscale intensity such as 0.75 - a legal html color name, e.g., red,
# blue, darkslategray

#### CONFIGURATION BEGINS HERE

# The default backend; one of GTK GTKAgg GTKCairo GTK3Agg GTK3Cairo
# CocoaAgg MacOSX Qt4Agg Qt5Agg TkAgg WX WXAgg Agg Cairo GDK PS PDF SVG
# Template.
# You can also deploy your own backend outside of matplotlib by
# referring to the module name (which must be in the PYTHONPATH) as
# 'module://my_backend'.
backend      : qt4agg

# If you are using the Qt4Agg backend, you can choose here
# to use the PyQt4 bindings or the newer PySide bindings to
# the underlying Qt4 toolkit.
#backend.qt4 : PyQt4        # PyQt4 | PySide

# Note that this can be overridden by the environment variable
# QT_API used by Enthought Tool Suite (ETS); valid values are
# "pyqt" and "pyside".  The "pyqt" setting has the side effect of
# forcing the use of Version 2 API for QString and QVariant.

# The port to use for the web server in the WebAgg backend.
# webagg.port : 8888

# If webagg.port is unavailable, a number of other random ports will
# be tried until one that is available is found.
# webagg.port_retries : 50

# When True, open the webbrowser to the plot that is shown
# webagg.open_in_browser : True

# When True, the figures rendered in the nbagg backend are created with
# a transparent background.
# nbagg.transparent : True

# if you are running pyplot inside a GUI and your backend choice
# conflicts, we will automatically try to find a compatible one for
# you if backend_fallback is True
#backend_fallback: True

#interactive  : False
#toolbar      : toolbar2   # None | toolbar2  ("classic" is deprecated)
#timezone     : UTC        # a pytz timezone string, e.g., US/Central or Europe/Paris

# Where your matplotlib data lives if you installed to a non-default
# location.  This is where the matplotlib fonts, bitmaps, etc reside
#datapath : /home/jdhunter/mpldata

### LINES
# See http://matplotlib.org/api/artist_api.html#module-matplotlib.lines for more
# information on line properties.
lines.linewidth   : 2.0     # line width in points
#lines.linestyle   : -       # solid line
#lines.color       : blue    # has no affect on plot(); see axes.color_cycle
#lines.marker      : None    # the default marker
#lines.markeredgewidth  : 0.5     # the line width around the marker symbol
#lines.markersize  : 6            # markersize, in points
#lines.dash_joinstyle : miter        # miter|round|bevel
#lines.dash_capstyle : butt          # butt|round|projecting
#lines.solid_joinstyle : miter       # miter|round|bevel
#lines.solid_capstyle : projecting   # butt|round|projecting
lines.antialiased : True         # render lines in antialised (no jaggies)

### PATCHES
# Patches are graphical objects that fill 2D space, like polygons or
# circles.  See
# http://matplotlib.org/api/artist_api.html#module-matplotlib.patches
# information on patch properties
#patch.linewidth        : 1.0     # edge width in points
#patch.facecolor        : blue
#patch.edgecolor        : black
#patch.antialiased      : True    # render patches in antialised (no jaggies)
patch.linewidth        : 0.5     # edge width in points
patch.facecolor        : a6cee3
patch.edgecolor        : eeeeee
patch.antialiased      : True    # render patches in antialised (no jaggies)

### FONT
#
# font properties used by text.Text.  See
# http://matplotlib.org/api/font_manager_api.html for more
# information on font properties.  The 6 font properties used for font
# matching are given below with their default values.
#
# The font.family property has five values: 'serif' (e.g., Times),
# 'sans-serif' (e.g., Helvetica), 'cursive' (e.g., Zapf-Chancery),
# 'fantasy' (e.g., Western), and 'monospace' (e.g., Courier).  Each of
# these font families has a default list of font names in decreasing
# order of priority associated with them.  When text.usetex is False,
# font.family may also be one or more concrete font names.
#
# The font.style property has three values: normal (or roman), italic
# or oblique.  The oblique style will be used for italic, if it is not
# present.
#
# The font.variant property has two values: normal or small-caps.  For
# TrueType fonts, which are scalable fonts, small-caps is equivalent
# to using a font size of 'smaller', or about 83% of the current font
# size.
#
# The font.weight property has effectively 13 values: normal, bold,
# bolder, lighter, 100, 200, 300, ..., 900.  Normal is the same as
# 400, and bold is 700.  bolder and lighter are relative values with
# respect to the current weight.
#
# The font.stretch property has 11 values: ultra-condensed,
# extra-condensed, condensed, semi-condensed, normal, semi-expanded,
# expanded, extra-expanded, ultra-expanded, wider, and narrower.  This
# property is not currently implemented.
#
# The font.size property is the default font size for text, given in pts.
# 12pt is the standard value.
#
#font.family         : sans-serif
font.family         : serif
#font.style          : normal
#font.variant        : normal
font.weight         : 100
#font.stretch        : normal
# note that font.size controls default text sizes.  To configure
# special text sizes tick labels, axes, labels, title, etc, see the rc
# settings for axes and ticks. Special text sizes can be defined
# relative to font.size, using the following values: xx-small, x-small,
# small, medium, large, x-large, xx-large, larger, or smaller
font.size           : 12.0
font.serif          : Bitstream Vera Serif, New Century Schoolbook, Century Schoolbook L, Utopia, ITC Bookman, Bookman, Nimbus Roman No9 L, Times New Roman, Times, Palatino, Charter, serif
#font.sans-serif     : Bitstream Vera Sans, Lucida Grande, Verdana, Geneva, Lucid, Arial, Helvetica, Avant Garde, sans-serif
#font.cursive        : Apple Chancery, Textile, Zapf Chancery, Sand, cursive
#font.fantasy        : Comic Sans MS, Chicago, Charcoal, Impact, Western, fantasy
#font.monospace      : Bitstream Vera Sans Mono, Andale Mono, Nimbus Mono L, Courier New, Courier, Fixed, Terminal, monospace

### TEXT
# text properties used by text.Text.  See
# http://matplotlib.org/api/artist_api.html#module-matplotlib.text for more
# information on text properties

#text.color          : black
text.color          : 555555

### LaTeX customizations. See http://www.scipy.org/Wiki/Cookbook/Matplotlib/UsingTex
#text.usetex         : False  # use latex for all text handling. The following fonts
                              # are supported through the usual rc parameter settings:
                              # new century schoolbook, bookman, times, palatino,
                              # zapf chancery, charter, serif, sans-serif, helvetica,
                              # avant garde, courier, monospace, computer modern roman,
                              # computer modern sans serif, computer modern typewriter
                              # If another font is desired which can loaded using the
                              # LaTeX \usepackage command, please inquire at the
                              # matplotlib mailing list
#text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling
                            # unicode strings.
#text.latex.preamble :  # IMPROPER USE OF THIS FEATURE WILL LEAD TO LATEX FAILURES
                            # AND IS THEREFORE UNSUPPORTED. PLEASE DO NOT ASK FOR HELP
                            # IF THIS FEATURE DOES NOT DO WHAT YOU EXPECT IT TO.
                            # preamble is a comma separated list of LaTeX statements
                            # that are included in the LaTeX document preamble.
                            # An example:
                            # text.latex.preamble : \usepackage{bm},\usepackage{euler}
                            # The following packages are always loaded with usetex, so
                            # beware of package collisions: color, geometry, graphicx,
                            # type1cm, textcomp. Adobe Postscript (PSSNFS) font packages
                            # may also be loaded, depending on your font settings

#text.dvipnghack : None      # some versions of dvipng don't handle alpha
                             # channel properly.  Use True to correct
                             # and flush ~/.matplotlib/tex.cache
                             # before testing and False to force
                             # correction off.  None will try and
                             # guess based on your dvipng version

#text.hinting : auto   # May be one of the following:
                       #   'none': Perform no hinting
                       #   'auto': Use freetype's autohinter
                       #   'native': Use the hinting information in the
                       #             font file, if available, and if your
                       #             freetype library supports it
                       #   'either': Use the native hinting information,
                       #             or the autohinter if none is available.
                       # For backward compatibility, this value may also be
                       # True === 'auto' or False === 'none'.
#text.hinting_factor : 8 # Specifies the amount of softness for hinting in the
                         # horizontal direction.  A value of 1 will hint to full
                         # pixels.  A value of 2 will hint to half pixels etc.

#text.antialiased : True # If True (default), the text will be antialiased.
                         # This only affects the Agg backend.

# The following settings allow you to select the fonts in math mode.
# They map from a TeX font name to a fontconfig font pattern.
# These settings are only used if mathtext.fontset is 'custom'.
# Note that this "custom" mode is unsupported and may go away in the
# future.
#mathtext.cal : cursive
#mathtext.rm  : serif
#mathtext.tt  : monospace
#mathtext.it  : serif:italic
#mathtext.bf  : serif:bold
#mathtext.sf  : sans
#mathtext.fontset : cm # Should be 'cm' (Computer Modern), 'stix',
                       # 'stixsans' or 'custom'
#mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern
                                 # fonts when a symbol can not be found in one of
                                 # the custom math fonts.

#mathtext.default : it # The default font to use for math.
                       # Can be any of the LaTeX font names, including
                       # the special name "regular" for the same font
                       # used in regular text.

### AXES
# default face and edge color, default tick sizes,
# default fontsizes for ticklabels, and so on.  See
# http://matplotlib.org/api/axes_api.html#module-matplotlib.axes


image.cmap      :   coolwarm    #use this colormap for surface plotting... I hope!
#axes.hold           : True    # whether to clear the axes by default on
#axes.facecolor      : white   # axes background color
axes.facecolor      : eeeeee   # axes background color
#axes.edgecolor      : black   # axes edge color
#axes.edgecolor      : bcbcbc   # axes edge color
axes.edgecolor      : 555555   # axes edge color
axes.linewidth      : 1.0     # edge linewidth
#axes.grid           : False   # display grid or not
axes.titlesize      : x-large   # fontsize of the axes title
axes.labelsize      : large  # fontsize of the x any y labels
#axes.labelweight    : normal  # weight of the x and y labels
axes.labelcolor     : 555555
axes.axisbelow      : True   # whether axis gridlines and ticks are below
                               # the axes elements (lines, text, etc)

#axes.formatter.limits : -7, 7 # use scientific notation if log10
                               # of the axis range is smaller than the
                               # first or larger than the second
#axes.formatter.use_locale : False # When True, format tick labels
                                   # according to the user's locale.
                                   # For example, use ',' as a decimal
                                   # separator in the fr_FR locale.
#axes.formatter.use_mathtext : False # When True, use mathtext for scientific
                                     # notation.
#axes.formatter.useoffset      : True    # If True, the tick label formatter
                                         # will default to labeling ticks relative
                                         # to an offset when the data range is very
                                         # small compared to the minimum absolute
                                         # value of the data.

#axes.unicode_minus  : True    # use unicode for the minus symbol
                               # rather than hyphen.  See
                               # http://en.wikipedia.org/wiki/Plus_and_minus_signs#Character_codes
#axes.color_cycle    : b, g, r, c, m, y, k  # color cycle for plot lines
                                            # as list of string colorspecs:
                                            # single letter, long name, or
                                            # web-style hex
#axes.color_cycle    : 3b4cc0, 5d7ce6, 82a6fb, aac7fd, cdd9ec, ead4c8, f7b89c, f18d6f, d95847, b40426  # A colorcycle generated from the 'coolwarm' colormap using the commands C = [matplotlib.cm.get_cmap('coolwarm')(k) for k in linspace(0,1,10)]; [matplotlib.colors.rgb2hex(C[k]) for k in range(0,10)]    
#axes.color_cycle    : 1b9e77, ad6e1d, a36861, 9b58a5, da367f, 739929, bba90b, c8920e, 97722e, 666666    # " 'Dark2 colormap 
#axes.color_cycle    : 0b0000, 540000, 9e0000, ea0000, ff3400, ff8000, ffca00, ffff22, ffff91, ffffff    # " 'hot' colormap
#axes.color_cycle    : ff0000, ff4000, ff8000, ffc000, ffff00 # " 'autumn' colormap
#axes.color_cycle    :   000000, 4f3220, 9e6440, ed9660, ffc77f # " 'copper' colormap
#axes.color_cycle    : 333399, 01cc66, fefe98, 815e56, ffffff # " 'terrain' colormap
#axes.color_cycle    : 000000, 6701db, 9309dd, b52000, d04c00, e99500, ffff00 #'gnuplot' colormap
#axes.color_cycle    : 000000, 0000a8, 4200ff, c92ad5, ff7e81, ffd42b, ffffff # 'gnuplot2' colormap
#axes.color_cycle    : a6cee3, 1f78b4, b2df8a, 33a02c, fb9a99, e31a1c #custom from colorbrewer2.org NOTE: PREFERRED LIGHT COLORS
#axes.color_cycle    : e41a1c, 377eb8, 4daf4a, 984ea3, ff7f00, ffff33, a65628
#axes.color_cycle    : 3e3d4c, 114488, 551188, 2b2b2b, 1104fb, 111011, 8888aa #Custom from palletton.com
#axes.color_cycle    : e34848, 2b8888, 3ab53a, d46f1c, d41c1c, 117f7f, 16a916, 6b3100, 004040, 6b0000, 005600 #palleton
axes.color_cycle    : 005757, e34848, e38e48, 2b8888, 3ab53a, 000000 #palleton
#axes.color_cycle    : a6cee3, 1f78b4, b2df8a, 33a02c #custom. colorblind safe

#axes.xmargin        : 0  # x margin.  See `axes.Axes.margins`
#axes.ymargin        : 0  # y margin See `axes.Axes.margins`

#polaraxes.grid      : True    # display grid on polar axes
#axes3d.grid         : True    # display grid on 3d axes

### TICKS
# see http://matplotlib.org/api/axis_api.html#matplotlib.axis.Tick
#xtick.major.size     : 4      # major tick size in points
#xtick.minor.size     : 2      # minor tick size in points
#xtick.major.width    : 0.5    # major tick width in points
#xtick.minor.width    : 0.5    # minor tick width in points
#xtick.major.pad      : 4      # distance to major tick label in points
#xtick.minor.pad      : 4      # distance to the minor tick label in points
#xtick.color          : k      # color of the tick labels
#xtick.labelsize      : medium # fontsize of the tick labels
#xtick.direction      : in     # direction: in, out, or inout

xtick.major.size     : 4      # major tick size in points
xtick.minor.size     : 4      # minor tick size in points
#xtick.major.width    : 0.5    # major tick width in points
#xtick.minor.width    : 0.5    # minor tick width in points
xtick.major.pad      : 6      # distance to major tick label in points
xtick.minor.pad      : 6      # distance to the minor tick label in points
xtick.color          : 555555      # color of the tick labels
xtick.labelsize      : small # fontsize of the tick labels
xtick.direction      : in     # direction: in, out, or inout

#ytick.major.size     : 4      # major tick size in points
#ytick.minor.size     : 2      # minor tick size in points
#ytick.major.width    : 0.5    # major tick width in points
#ytick.minor.width    : 0.5    # minor tick width in points
#ytick.major.pad      : 4      # distance to major tick label in points
#ytick.minor.pad      : 4      # distance to the minor tick label in points
#ytick.color          : k      # color of the tick labels
#ytick.labelsize      : medium # fontsize of the tick labels
#ytick.direction      : in     # direction: in, out, or inout

ytick.major.size     : 4      # major tick size in points
ytick.minor.size     : 4      # minor tick size in points
#ytick.major.width    : 0.5    # major tick width in points
#ytick.minor.width    : 0.5    # minor tick width in points
ytick.major.pad      : 6      # distance to major tick label in points
ytick.minor.pad      : 6      # distance to the minor tick label in points
ytick.color          : 555555      # color of the tick labels
ytick.labelsize      : small # fontsize of the tick labels
ytick.direction      : in     # direction: in, out, or inout


### GRIDS
#grid.color       :   black   # grid color
#grid.linestyle   :   :       # dotted
#grid.linewidth   :   0.5     # in points
#grid.alpha       :   1.0     # transparency, between 0.0 and 1.0

### Legend
#legend.fancybox      : False  # if True, use a rounded box for the
                               # legend, else a rectangle
#legend.isaxes        : True
#legend.numpoints     : 2      # the number of points in the legend line
legend.fontsize      : small
#legend.borderpad     : 0.5    # border whitespace in fontsize units
#legend.markerscale   : 1.0    # the relative size of legend markers vs. original
# the following dimensions are in axes coords
#legend.labelspacing  : 0.5    # the vertical space between the legend entries in fraction of fontsize
#legend.handlelength  : 2.     # the length of the legend lines in fraction of fontsize
#legend.handleheight  : 0.7     # the height of the legend handle in fraction of fontsize
#legend.handletextpad : 0.8    # the space between the legend line and legend text in fraction of fontsize
#legend.borderaxespad : 0.5   # the border between the axes and legend edge in fraction of fontsize
#legend.columnspacing : 2.    # the border between the axes and legend edge in fraction of fontsize
#legend.shadow        : False
#legend.frameon       : True   # whether or not to draw a frame around legend
#legend.framealpha    : None    # opacity of of legend frame
#legend.scatterpoints : 3 # number of scatter points

### FIGURE
# See http://matplotlib.org/api/figure_api.html#matplotlib.figure.Figure
figure.figsize   : 10, 11    # figure size in inches
#figure.dpi       : 80      # figure dots per inch
#figure.facecolor : 0.75    # figure facecolor; 0.75 is scalar gray
figure.facecolor : 0.85    # figure facecolor; 0.75 is scalar gray
#figure.edgecolor : white   # figure edgecolor
figure.edgecolor : 0.5   # figure edgecolor
figure.autolayout : True  # When True, automatically adjust subplot
                            # parameters to make the plot fit the figure
#figure.max_open_warning : 20  # The maximum number of figures to open through
                               # the pyplot interface before emitting a warning.
                               # If less than one this feature is disabled.

# The figure subplot parameters.  All dimensions are a fraction of the
# figure width or height
#figure.subplot.left    : 0.125  # the left side of the subplots of the figure
#figure.subplot.right   : 0.9    # the right side of the subplots of the figure
#figure.subplot.bottom  : 0.1    # the bottom of the subplots of the figure
#figure.subplot.top     : 0.9    # the top of the subplots of the figure
#figure.subplot.wspace  : 0.2    # the amount of width reserved for blank space between subplots
figure.subplot.hspace  : 0.5    # the amount of height reserved for white space between subplots

### IMAGES
#image.aspect : equal             # equal | auto | a number
#image.interpolation  : bilinear  # see help(imshow) for options
#image.cmap   : jet               # gray | jet etc...
#image.lut    : 256               # the size of the colormap lookup table
#image.origin : upper             # lower | upper
#image.resample  : False

### CONTOUR PLOTS
#contour.negative_linestyle :  dashed # dashed | solid

### Agg rendering
### Warning: experimental, 2008/10/10
#agg.path.chunksize : 0           # 0 to disable; values in the range
                                  # 10000 to 100000 can improve speed slightly
                                  # and prevent an Agg rendering failure
                                  # when plotting very large data sets,
                                  # especially if they are very gappy.
                                  # It may cause minor artifacts, though.
                                  # A value of 20000 is probably a good
                                  # starting point.
### SAVING FIGURES
#path.simplify : True   # When True, simplify paths by removing "invisible"
                        # points to reduce file size and increase rendering
                        # speed
#path.simplify_threshold : 0.1  # The threshold of similarity below which
                                # vertices will be removed in the simplification
                                # process
#path.snap : True # When True, rectilinear axis-aligned paths will be snapped to
                  # the nearest pixel when certain criteria are met.  When False,
                  # paths will never be snapped.
#path.sketch : None # May be none, or a 3-tuple of the form (scale, length,
                    # randomness).
                    # *scale* is the amplitude of the wiggle
                    # perpendicular to the line (in pixels).  *length*
                    # is the length of the wiggle along the line (in
                    # pixels).  *randomness* is the factor by which
                    # the length is randomly scaled.

# the default savefig params can be different from the display params
# e.g., you may want a higher resolution, or to make the figure
# background white
#savefig.dpi         : 100      # figure dots per inch
#savefig.facecolor   : white    # figure facecolor when saving
#savefig.edgecolor   : white    # figure edgecolor when saving
#savefig.format      : png      # png, ps, pdf, svg
#savefig.bbox        : standard # 'tight' or 'standard'.
                                # 'tight' is incompatible with pipe-based animation
                                # backends but will workd with temporary file based ones:
                                # e.g. setting animation.writer to ffmpeg will not work,
                                # use ffmpeg_file instead
#savefig.pad_inches  : 0.1      # Padding to be used when bbox is set to 'tight'
#savefig.jpeg_quality: 95       # when a jpeg is saved, the default quality parameter.
#savefig.directory   : ~        # default directory in savefig dialog box,
                                # leave empty to always use current working directory
#savefig.transparent : False    # setting that controls whether figures are saved with a
                                # transparent background by default

# tk backend params
#tk.window_focus   : False    # Maintain shell focus for TkAgg

# ps backend params
#ps.papersize      : letter   # auto, letter, legal, ledger, A0-A10, B0-B10
#ps.useafm         : False    # use of afm fonts, results in small files
#ps.usedistiller   : False    # can be: None, ghostscript or xpdf
                                          # Experimental: may produce smaller files.
                                          # xpdf intended for production of publication quality files,
                                          # but requires ghostscript, xpdf and ps2eps
#ps.distiller.res  : 6000      # dpi
#ps.fonttype       : 3         # Output Type 3 (Type3) or Type 42 (TrueType)

# pdf backend params
#pdf.compression   : 6 # integer from 0 to 9
                       # 0 disables compression (good for debugging)
#pdf.fonttype       : 3         # Output Type 3 (Type3) or Type 42 (TrueType)

# svg backend params
#svg.image_inline : True       # write raster image data directly into the svg file
#svg.image_noscale : False     # suppress scaling of raster data embedded in SVG
#svg.fonttype : 'path'         # How to handle SVG fonts:
#    'none': Assume fonts are installed on the machine where the SVG will be viewed.
#    'path': Embed characters as paths -- supported by most SVG renderers
#    'svgfont': Embed characters as SVG fonts -- supported only by Chrome,
#               Opera and Safari

# docstring params
#docstring.hardcopy = False  # set this when you want to generate hardcopy docstring

# Set the verbose flags.  This controls how much information
# matplotlib gives you at runtime and where it goes.  The verbosity
# levels are: silent, helpful, debug, debug-annoying.  Any level is
# inclusive of all the levels below it.  If your setting is "debug",
# you'll get all the debug and helpful messages.  When submitting
# problems to the mailing-list, please set verbose to "helpful" or "debug"
# and paste the output into your report.
#
# The "fileo" gives the destination for any calls to verbose.report.
# These objects can a filename, or a filehandle like sys.stdout.
#
# You can override the rc default verbosity from the command line by
# giving the flags --verbose-LEVEL where LEVEL is one of the legal
# levels, e.g., --verbose-helpful.
#
# You can access the verbose instance in your code
#   from matplotlib import verbose.
#verbose.level  : silent      # one of silent, helpful, debug, debug-annoying
#verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.stderr

# Event keys to interact with figures/plots via keyboard.
# Customize these settings according to your needs.
# Leave the field(s) empty if you don't need a key-map. (i.e., fullscreen : '')

#keymap.fullscreen : f               # toggling
#keymap.home : h, r, home            # home or reset mnemonic
#keymap.back : left, c, backspace    # forward / backward keys to enable
#keymap.forward : right, v           #   left handed quick navigation
#keymap.pan : p                      # pan mnemonic
#keymap.zoom : o                     # zoom mnemonic
#keymap.save : s                     # saving current figure
#keymap.quit : ctrl+w, cmd+w         # close the current figure
#keymap.grid : g                     # switching on/off a grid in current axes
#keymap.yscale : l                   # toggle scaling of y-axes ('log'/'linear')
#keymap.xscale : L, k                # toggle scaling of x-axes ('log'/'linear')
#keymap.all_axes : a                 # enable all axes

# Control location of examples data files
#examples.directory : ''   # directory to look in for custom installation

###ANIMATION settings
#animation.writer : ffmpeg         # MovieWriter 'backend' to use
#animation.codec : mpeg4           # Codec to use for writing movie
#animation.bitrate: -1             # Controls size/quality tradeoff for movie.
                                   # -1 implies let utility auto-determine
#animation.frame_format: 'png'     # Controls frame format used by temp files
#animation.ffmpeg_path: 'ffmpeg'   # Path to ffmpeg binary. Without full path
                                   # $PATH is searched
#animation.ffmpeg_args: ''         # Additional arguments to pass to ffmpeg
#animation.avconv_path: 'avconv'   # Path to avconv binary. Without full path
                                   # $PATH is searched
#animation.avconv_args: ''         # Additional arguments to pass to avconv
#animation.mencoder_path: 'mencoder'
                                   # Path to mencoder binary. Without full path
                                   # $PATH is searched
#animation.mencoder_args: ''       # Additional arguments to pass to mencoder
#animation.convert_path: 'convert' # Path to ImageMagick's convert binary.
                                   # On Windows use the full path since convert
                                   # is also the name of a system tool.
